One of the important properties of hydrocarbon fuel is the <i><u>ignition delay time</u></i>. For automobiles, the gasoline is graded by the <i>octane number</i> which is closely related to the fuel's ignition characteristics. Higher octane number fuel tends to ignite more easily. However, this does not mean you should blindly put the highest octane number fuel into your car. Car engines are designed for specific operating conditions, and you should use the gasoline grades recommended by the car/engine manufacturer to prevent damages to the engine.   

This tutorial employs the <i><u>constant-pressure batch reactor model</u></i> (with the energy equation <u>ON</u>) to predict the <u>auto-ignition delay time</u> of a <em>Primary Reference Fuel (PRF)</em>. PRF is created for automobile fuel researches and consists of two major components: n-heptane $C_{7}H_{16}$ and iso-octane $C_{8}H_{18}$. By definition, n-heptane is assigned with an octane number of 0, and an octane number of 100 for iso-octane. Thus, a PRF of 20% n-heptane and 80% iso-octane has an octance number of 80. In this tutorial, a PRF 60 fuel is used to show the <i>negative temperature coefficient (NTC)</i> behavior in the ignition delay curve. The ignition delay curve is constructed by collecting the predicted auto-ignition delay times with various initial gas conditions. Here, the initial gas temperature is increased from 700K to 1080K.   

You start with importing these required Python packages: <code>os</code>, <code>chemkin</code>, <code>numppy</code>, and <code>matplotlib</code> (for plotting purposes). In addition, you will have to import the <code>chemkin.batchreactor</code> module to access all the batch reactor models in <strong>PyChemkin</strong> and the <code>time</code> package for calculating the total runtime.  

In [ ]:
import os
import time 
import chemkin as ck                                 # Chemkin
from chemkin.batchreactor import *                   # chemkin batch reactor models (transient)
%matplotlib inline
import matplotlib.pyplot as plt                                 # plotting
import numpy as np                                              # number crunching

# check working directory
current_dir = os.getcwd()
print('current working directory: ' + current_dir)
# set verbose mode
ck.verbose = True

<h5>Set up the <em>Chemistry Set</em></h5>Create and preprocess the <em>Chemistry</em> set object for the current PyChemkin project. For PRF, the encrypted 14-component gasoline mechanism, gasoline_14comp_WBencrypted.inp, is used. The <em>Chemistry set</em> is named <code>gasoline</code>. Note that this gasoline mechanism does not come with any transport data so you do not need to provide the transport data file. 

In [ ]:
# set mechanism directory (the default chemkin mechanism data directory)
data_dir = ck.ansys_dir + '\\reaction\\data'
mechanism_dir = data_dir
# create a chemistry set based on GRI 3.0
gasoline = ck.Chemistry(label='gasoline 14comp')
# set mechanism input files
# inclusion of the full file path is recommended
gasoline.chemfile = mechanism_dir+'\\gasoline_14comp_WBencrypt.inp'
# preprocess the mechanism files
iError = gasoline.preprocess()

<h5>Define the fuel-air mixture</h5>The PRF 60 is a mixture of 60% iso-octane and 40% n-heptane by volume so your fuel recipe will have two entries and will be assigned as the mixture mole fractions <code>fuelmixture.X</code>. Use the <i><u>equivalence ratio</u></i> method to create a stoiciometric fuel-air mixture <code>premixed</code>.

In [ ]:
# create a premixed fuel-oxidizer mixture by assigning the equivalence ratio
# create the fuel mixture
fuelmixture = ck.Mixture(gasoline)
# set fuel = composition PRF 60 
fuelmixture.X = [('ic8h18', 0.6), ('nc7h16', 0.4)]
# setting pressure and temperature 
fuelmixture.pressure = 5.0 * ck.Patm
fuelmixture.temperature = 1500.0
# create the oxidizer mixture: air
air = ck.Mixture(gasoline)
air.X = [('o2', 0.21), ('n2', 0.79)]
# setting pressure and temperature (the same as the fuel mixture)
air.pressure = fuelmixture.pressure
air.temperature = fuelmixture.temperature
# create the premixed mixture to be defined
premixed = ck.Mixture(gasoline)
# products from the complete combustion of the fuel mixture and air
products = ['co2', 'h2o', 'n2']
# species mole fractions of added/inert mixture. can also create an additives mixture here
add_frac = np.zeros(gasoline.KK, dtype=np.double)   # no additives: all zeros
iError = premixed.XbyEquivalenceRatio(gasoline, fuelmixture.X, air.X, add_frac, products, equivalenceratio=1.0)
if iError != 0:
    raise RuntimeError

List the composition of <code>premixed</code> so you can verify the initial gas composition in the batch reactor. 

In [ ]:
# list the composition of the premixed mixture
premixed.listcomposition(mode='mole')

Set the temperature (700K) and the pressure (40 atm) of <code>premixed</code>. These values will also be used to set the initial conditions of the batch reactor.  

In [ ]:
# set mixture temperature and pressure (equivalent to setting the initial temperature and pressure of the reactor)
premixed.pressure = 40.0 * ck.Patm
premixed.temperature = 700.0 

<h5>Create the Batch Reactor Object</h5>Instantiate a <code>GivenPressureBatchReactor_EnergyConservation</code> batch reactor object <code>MyCONP</code>. <strong>PyChemkin</strong> requires every <em>Batch Reactor</em> object to be associated with a <em>Mixture</em>. The <em>Mixture</em> implicitly links the <em>Chemistry Set</em> (gas-phase mechanism and properties) to the batch reactor object. Additionally, it also defines the initial conditions (pressure, temperature, volume, and gas composition) of the reactor. 

In [ ]:
#
# create a constant pressure batch reactor (with energy equation)
#
MyCONP = GivenPressureBatchReactor_EnergyConservation(premixed, label='CONP')

<h5>Set/Verify Reactor Initial Conditions</h5>List the initial gas composition inside the batch reactor <code>MyCONP</code>. You can compare this list to the <code>premixed</code> composition earlier to verify that they are exactly the same.  

In [ ]:
# show initial gas composition inside the reactor
MyCONP.listcomposition(mode='mole')

Since no volume is given to <code>premixed</code>, you have to specify the reactor volume here. 

In [ ]:
# reactor volume [cm3]
MyCONP.volume = 10.0 

Provide a simulation end time [sec] that is long enough to allow the gas mixture to auto-ignition for all cases. 

In [ ]:
# simulation end time [sec]
MyCONP.time = 1.0 

<h5>Set Output Control Parameters</h5>

By default, time intervals for both print and save solution are 100th of the simulation end time. In this case $$dt=time/100=0.001.$$ You can change them to different values.

In [ ]:
# output controls
# set timestep between saving solution
MyCONP.timestepforsavingsolution = 0.001
# change timestep between saving solution
MyCONP.timestepforsavingsolution = 0.01

You can turn on the adaptive solution saving to resolve the steep variations in the solution profile. Here additional solution data point will be saved for every <u>100K</u> change in gas <u>temperature</u>.  

In [ ]:
# turn ON adaptive solution saving
MyCONP.adaptivesolutionsaving(mode=True, value_change=100, target='TEMPERATURE')

<h5>Set Solver Parameters</h5>Changing solver tolerances sometimes is necessary; especially when you expect large and sudden variation in the solution. 

In [ ]:
# set tolerance
MyCONP.settolerances(absolute_tolerance=1.0e-10, relative_tolerance=1.0e-8)

Verify your changes to the solver parameters.

In [ ]:
# show solver option
print(f'timestep between solution printing: {MyCONP.timestepforprintingsolution}')
# show timestep between printing solution
print(f'forced non-negative solution values: {MyCONP.forcenonnegative}')

<h5>Define Auto-Ignition</h5>The <code>batch reactor</code> looks for the inflection point in the gas temperature profile as the indication of an auto-ignition. You can choose a different auto-ignition definition.  

Use the <code>showignitiondefinition</code> method to see all available options.

In [ ]:
# set ignition delay
ck.showignitiondefinition()

The default temperature inflection point definition is retained.

In [ ]:
MyCONP.setignitiondelay(method='T_inflection')

You can force the simulation to terminate when the gas is auto-ignited.  

In [ ]:
# stop after ignition is detected (not recommended for ignition delay time calculations)
# MyCONP.stopafterignition()

<h5>Set Up the Parameter Study</h5>Construct the ignition delay curve by varying the initial reactor temperature in 20K increments from 700K to 1080K.

In [ ]:
#
# loop over initial reactor temperature to create an ignition delay time plot
#
npoints = 20
delta_temp = 20.0 
init_temp = premixed.temperature
delaytime = np.zeros(npoints,dtype=np.double)
temp_inv = np.zeros_like(delaytime, dtype=np.double)

Keep the <u>wall time</u> at the start of the simulations so you can estimate the total runtime in seconds. 

In [ ]:
# set the start wall time
start_time = time.time()

<h5>Run the Simulations</h5>Loop over all cases one by one and save the inverse of gas temperature and the predicted ignition delay time for post-processing.

In [ ]:
# loop over all cases with different initial gas/reactor temperatures
for i in range(npoints):
    # update the initial reactor temperature
    MyCONP.temperature = init_temp  # K
    # show the additional keywords given by user (verify inputs before running the simulation)
    # MyCONP.showkeywordinputlines()
    # run the reactor model
    runstatus = MyCONP.run()
    #
    if runstatus == 0:
        # plot 1/T instead of T 
        temp_inv[i] = 1.0e0/init_temp
        # get ignition delay time
        delaytime[i] = MyCONP.getignitiondelay()
        print(f'ignition delay time = {delaytime[i]} [msec]') 
    else:
        # if get this, most likely the END time is too short 
        print(Color.RED + '>>> RUN FAILED <<<', end='\n' + Color.END)
    init_temp += delta_temp

Calculate the total runtime far all 20 runs.

In [ ]:
# compute the total runtime
runtime = time.time() - start_time
print(f"total simulation duration: {runtime} [sec] for {npoints} cases")

<h5>Plot Ignition Delay Curve</h5>The ignition delay curve is customarily displayed as ignition delay time vs. inversed temperature. 

In [ ]:
# create an ignition delay versus 1/T plot for the PRF fuel (should exhibit the NTC region)
plt.rcParams.update({'figure.autolayout': True})
fig, ax1 =plt.subplots()
ax1.semilogy(temp_inv,delaytime,'bs--')
ax1.set_xlabel('1/T [1/K]')
ax1.set_ylabel('Ignition delay time [msec]')
# Create a secondary x-axis for T (=1/(1/T))
def one_over(x):
    """
    Vectorized 1/x, treating x==0 manually 
    (from the matplotlib website)
    """
    x = np.array(x, np.double)
    near_zero = np.isclose(x, 0.0e0)
    x[near_zero] = np.inf
    x[~near_zero] = 1.0e0 / x[~near_zero]
    return x
# the function "1/x" is its own inverse
inverse = one_over
ax2 = ax1.secondary_xaxis('top', functions=(one_over, inverse))
ax2.set_xlabel('T [K]')
plt.show()


Intuitively, you expect that the reactiveity increases as the temperature increases. However, you can see that, in this case, the ignition delay curve does not vary monotonically with the temperature. Between 850K and 950K, the ignition delay time actually increases slightly (because the reactivity decreases). This is regarded as the <i><u>Negative Temperature Coefficient (NTC)</u></i> behavior which is often observed during low-temperature oxidation of large hydrocarbon fuels.    